In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

In [4]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
#model_name = "elyza/ELYZA-japanese-Llama-2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.25it/s]


In [5]:
if torch.cuda.is_available():
    model = model.to("cuda")
    print("cuda", flush=True)

cuda


In [6]:
def make_prompt(text: str) -> str:
    return "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )

In [7]:
text = (
    "以下の文を要約しなさい。"
    "要約文だけを出力しなさい。余計な文章はつけないでください。"
    "「クマが侵入し従業員が襲われた秋田市土崎港のスーパーで、クマを運び出した後に売り場を確認したところ精肉や総菜コーナーが荒らされていましたがパックに詰められた肉は食べられていないことが新たに分かりました。秋田市の農地森林整備課などによりますと、クマが侵入した秋田市土崎港のスーパーで２日ドローンを使って店内を調査しました。売り場の入口近くにあった花びんが割れていたほか売り場の精肉や総菜コーナーの商品が荒らされていたということです。一方で、パックに詰められた精肉を食べた形跡は確認されなかったということです。店に侵入したメスのクマは２日に市などが箱わなで捕獲しています。秋田市は店が設置した防犯カメラなどから、クマの侵入経路などの確認を続けています。」"
)

In [8]:
print(text)

以下の文を要約しなさい。要約文だけを出力しなさい。余計な文章はつけないでください。「クマが侵入し従業員が襲われた秋田市土崎港のスーパーで、クマを運び出した後に売り場を確認したところ精肉や総菜コーナーが荒らされていましたがパックに詰められた肉は食べられていないことが新たに分かりました。秋田市の農地森林整備課などによりますと、クマが侵入した秋田市土崎港のスーパーで２日ドローンを使って店内を調査しました。売り場の入口近くにあった花びんが割れていたほか売り場の精肉や総菜コーナーの商品が荒らされていたということです。一方で、パックに詰められた精肉を食べた形跡は確認されなかったということです。店に侵入したメスのクマは２日に市などが箱わなで捕獲しています。秋田市は店が設置した防犯カメラなどから、クマの侵入経路などの確認を続けています。」


In [9]:
prompt = make_prompt(text)

In [10]:
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=2000,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output, flush=True)

クマが侵入し従業員が襲われた秋田市のスーパーで、精肉や総菜コーナーが荒らされたが、パックに詰められた肉は食べられていないことが分かりました。
